# Lab 4: Macros và Jinja - Code Reuse trong dbt

## 🎯 Objectives
- Hiểu macros là gì và tại sao sử dụng
- Tạo custom macros
- Sử dụng Jinja templating nâng cao
- Sử dụng dbt_utils package
- Code reuse patterns
- Best practices cho macros

## 📋 Prerequisites
- Hoàn thành Lab 1-3
- Hiểu về Jinja templating cơ bản
- Models đã được tạo

## 🏗️ Macros Overview

**Macros** là reusable code blocks trong dbt:
- Được định nghĩa trong `macros/` directory
- Sử dụng Jinja syntax
- Có thể nhận parameters
- Có thể được gọi từ models, tests, hoặc macros khác

### Macro Benefits:
- **DRY (Don't Repeat Yourself)**: Reuse code
- **Consistency**: Standardize transformations
- **Maintainability**: Update một chỗ, apply everywhere
- **Power**: Complex logic có thể được abstracted


## 1. Tạo Custom Macro

Tạo macro đơn giản để convert cents sang dollars.


In [ ]:
# Tạo example macro
macro_example = """
-- macros/cents_to_dollars.sql
{% macro cents_to_dollars(column_name, scale=2) %}
    round({{ column_name }} / 100.0, {{ scale }})
{% endmacro %}
"""

print("📝 Macro Example: cents_to_dollars")
print("=" * 60)
print(macro_example)
print("=" * 60)

print("\n💡 Macro Structure:")
print("  - {% macro macro_name(params) %}")
print("  - Macro body với Jinja")
print("  - {% endmacro %}")

# Tạo file
macros_dir = project_root / 'macros'
macros_dir.mkdir(exist_ok=True)

macro_file = macros_dir / 'cents_to_dollars.sql'
if not macro_file.exists():
    with open(macro_file, 'w') as f:
        f.write(macro_example.strip())
    print(f"\n✅ Created: {macro_file}")
else:
    print(f"\n✅ File already exists: {macro_file}")

print("\n📖 Usage trong model:")
print("  select {{ cents_to_dollars('amount_cents') }} as amount_dollars")


## 2. Advanced Macros

Macros với logic phức tạp hơn.


In [ ]:
# Advanced macro examples
advanced_macros = """
-- macros/generate_surrogate_key.sql
{% macro generate_surrogate_key(field_list) %}
    {%- set fields = [] -%}
    {%- for field in field_list -%}
        {%- set _ = fields.append(
            \"coalesce(cast(\" ~ field ~ \" as varchar), '')\"
        ) -%}
    {%- endfor -%}
    {{ dbt_utils.generate_surrogate_key(fields) }}
{% endmacro %}

-- macros/date_spine.sql
{% macro date_spine(
    datepart=\"day\",
    start_date=\"cast('2020-01-01' as date)\",
    end_date=\"cast('2021-01-01' as date)\"
) %}
    {{ dbt_utils.date_spine(
        datepart=datepart,
        start_date=start_date,
        end_date=end_date
    ) }}
{% endmacro %}

-- macros/pivot.sql
{% macro pivot(column, values, agg='sum', then_value='amount', else_value=0) %}
    {%- for value in values -%}
        {{ agg }}(
            case when {{ column }} = '{{ value }}' 
            then {{ then_value }} 
            else {{ else_value }} 
            end
        ) as {{ value }}{% if not loop.last %},{% endif %}
    {%- endfor -%}
{% endmacro %}
"""

print("🔧 Advanced Macro Examples:")
print("=" * 60)
print(advanced_macros)
print("=" * 60)

print("\n💡 Advanced Features:")
print("  - Loops với {% for %}")
print("  - Conditionals với {% if %}")
print("  - Variables với {% set %}")
print("  - Calling other macros")
print("  - Default parameters")


## 3. dbt_utils Package

dbt_utils cung cấp nhiều useful macros.


In [ ]:
print("📦 dbt_utils Package Macros:")
print("=" * 60)
print("""
1. dbt_utils.generate_surrogate_key()
   - Tạo surrogate key từ multiple columns
   - Usage: {{ dbt_utils.generate_surrogate_key(['customer_id', 'order_date']) }}

2. dbt_utils.date_spine()
   - Tạo date spine cho date ranges
   - Usage: {{ dbt_utils.date_spine(datepart='day', start_date='2024-01-01', end_date='2024-12-31') }}

3. dbt_utils.star()
   - Select all columns except specified ones
   - Usage: {{ dbt_utils.star(from=ref('customers'), except=['sensitive_col']) }}

4. dbt_utils.get_column_values()
   - Get distinct values từ column
   - Usage: {{ dbt_utils.get_column_values(table=ref('orders'), column='status') }}

5. dbt_utils.union_relations()
   - Union multiple relations
   - Usage: {{ dbt_utils.union_relations([ref('table1'), ref('table2')]) }}

6. dbt_utils.surrogate_key()
   - Simple surrogate key generation
   - Usage: {{ dbt_utils.surrogate_key(['col1', 'col2']) }}
""")
print("=" * 60)

print("\n💡 To use dbt_utils:")
print("  1. Add to packages.yml:")
print("     packages:")
print("       - package: dbt-labs/dbt_utils")
print("         version: 1.1.1")
print("  2. Run: dbt deps")
print("  3. Use macros trong models")


## 4. Jinja Templating nâng cao

Jinja features nâng cao cho complex logic.


In [ ]:
print("🔧 Advanced Jinja Features:")
print("=" * 60)
print("""
1. Variables:
   {% set my_var = 'value' %}
   {% set my_list = ['a', 'b', 'c'] %}
   {% set my_dict = {'key': 'value'} %}

2. Filters:
   {{ column_name | upper }}
   {{ column_name | lower }}
   {{ column_name | trim }}
   {{ value | default('N/A') }}

3. Loops:
   {% for item in list %}
       {{ item }}
   {% endfor %}
   
   {% for key, value in dict.items() %}
       {{ key }}: {{ value }}
   {% endfor %}

4. Conditionals:
   {% if condition %}
       ...
   {% elif other_condition %}
       ...
   {% else %}
       ...
   {% endif %}

5. Whitespace Control:
   {%- ... -%}  # Strip whitespace
   {{- ... -}}  # Strip whitespace

6. Comments:
   {# This is a comment #}

7. String Concatenation:
   {{ 'prefix_' ~ column_name ~ '_suffix' }}

8. List Operations:
   {{ list | length }}
   {{ list | first }}
   {{ list | last }}
   {{ list | join(', ') }}
""")
print("=" * 60)


## 5. Tóm tắt và Next Steps

### ✅ Những gì đã học:
1. Tạo custom macros
2. Advanced macros với logic phức tạp
3. Sử dụng dbt_utils package
4. Jinja templating nâng cao
5. Code reuse patterns

### 📚 Next Lab:
- **Lab 5**: Airflow Integration
- Tích hợp dbt với Airflow
- dbt operators trong Airflow
- Scheduling dbt runs

### 💡 Key Takeaways:

**Macros:**
- Reusable code blocks
- DRY principle
- Consistency và maintainability
- Complex logic abstraction

**Jinja:**
- Variables, loops, conditionals
- Filters và string operations
- Whitespace control
- Powerful templating language

**dbt_utils:**
- Useful macros package
- Common transformations
- Install với packages.yml
- Run `dbt deps` để install

### 🔗 Useful Links:
- [dbt Macros](https://docs.getdbt.com/docs/build/jinja-macros)
- [Jinja Documentation](https://jinja.palletsprojects.com/)
- [dbt_utils Package](https://github.com/dbt-labs/dbt-utils)
